# Breast Cancer Prediction 
_**Predict Breast Cancer using SageMaker's Linear-Learner with features derived from images of Breast Mass**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)
1. [Predict](#Predict)
1. [Extensions](#Extensions)

---

## Background
This notebook illustrates how one can use SageMaker's algorithms for solving applications which require `linear models` for prediction. For this illustration, we have taken an example for breast cancer prediction using UCI'S breast cancer diagnostic data set available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. The data set will be used to illustrate

* Basic setup for using SageMaker.
* converting datasets to protobuf format used by the Amazon SageMaker algorithms and uploading to S3. 
* Training SageMaker's linear learner on the data set.
* Hosting the trained model.
* Scoring using the trained model.



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give learning and hosting access to your data. The snippet below will use the same role used by your SageMaker notebook instance, if you're using other.  Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [1]:
import os
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

bucket = 'breast-cancer-prediction'# enter your s3 bucket where you will copy data and model artifacts
prefix = 'sagemaker/DEMO-breast-cancer-prediction' # place to upload training files within the bucket

Now we'll import the Python libraries we'll need.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

---
## Data

Data Source: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
        https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [3]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header = None)

# specify columns extracted from wbdc.names
data.columns = ["id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean",
                "compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean",
                "radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se",
                "concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst",
                "perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst",
                "concave points_worst","symmetry_worst","fractal_dimension_worst"] 

# save the data
data.to_csv("data.csv", sep=',', index=False)

# print the shape of the data file
print(data.shape)




(569, 32)


In [4]:
# show the top few rows
display(data.head())

# describe the data object
display(data.describe())

# we will also summarize the categorical field diganosis 
display(data.diagnosis.value_counts())

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


B    357
M    212
Name: diagnosis, dtype: int64

#### Key observations:
* Data has 569 observations and 32 columns.
* First field is 'id'.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features available for prediction.

## Create Features and Labels
#### Split the data into 80% training, 10% validation and 10% testing.

In [9]:
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
test_list = rand_split >= 0.9

In [17]:
data_train = data[train_list]
data_val = data[val_list]
data_test = data[test_list]

In [19]:
data_train.shape,data_val.shape,data_test.shape

((460, 32), (53, 32), (56, 32))

In [29]:


train_y = ((data_train.iloc[:,1] == 'M') +0).as_matrix();
train_X = data_train.iloc[:,2:].as_matrix();

val_y = ((data_val.iloc[:,1] == 'M') +0).as_matrix();
val_X = data_val.iloc[:,2:].as_matrix();

test_y = ((data_test.iloc[:,1] == 'M') +0).as_matrix();
test_X = data_test.iloc[:,2:].as_matrix();

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packa

In [31]:
data_train.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
4,84358402,M,20.29,14.34,135.1,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.2,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
6,844359,M,18.25,19.98,119.6,1040.0,0.09463,0.10900,0.1127,0.07400,...,22.88,27.66,153.2,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368


Now, we'll convert the datasets to the recordIO-wrapped protobuf format used by the Amazon SageMaker algorithms, and then upload this data to S3.  We'll start with training data.

In [35]:
train_file = 'linear_train.data'
train_file
io.BytesIO

'linear_train.data'

In [43]:
my_region = boto3.session.Session().region_name # set the region of the instance
my_region

'us-east-2'

In [46]:

bucket = 'breastcancer-saiprediction'# enter your s3 bucket where you will copy data and model artifacts
prefix = 'sagemaker/DEMO-breast-cancer-prediction' # place to upload training files within the bucket

s3 = boto3.resource('s3')
s3.create_bucket(Bucket=bucket,CreateBucketConfiguration={ 'LocationConstraint': my_region })

s3.Bucket(name='breastcancer-saiprediction')

In [47]:
train_file = 'linear_train.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype('float32'), train_y.astype('float32'))
f.seek(0)
# create an s3 bucket ,in that path store the data sa training file
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', train_file)).upload_fileobj(f)


Next we'll convert and upload the validation dataset.

In [48]:
validation_file = 'linear_validation.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype('float32'), val_y.astype('float32'))
f.seek(0)

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', validation_file)).upload_fileobj(f)

---
## Train

Now we can begin to specify our linear model.  Amazon SageMaker's Linear Learner actually fits many models in parallel, each with slightly different hyperparameters, and then returns the one with the best fit.  This functionality is automatically enabled.  We can influence this using parameters like:

- `num_models` to increase to total number of models run.  The specified parameters will always be one of those models, but the algorithm also chooses models with nearby parameter values in order to find a solution nearby that may be more optimal.  In this case, we're going to use the max of 32.
- `loss` which controls how we penalize mistakes in our model estimates.  For this case, let's use absolute loss as we haven't spent much time cleaning the data, and absolute loss will be less sensitive to outliers.
- `wd` or `l1` which control regularization.  Regularization can prevent model overfitting by preventing our estimates from becoming too finely tuned to the training data, which can actually hurt generalizability.  In this case, we'll leave these parameters as their default "auto" though.

### Specify container images used for training and hosting SageMaker's linear-learner

In [49]:
# See 'Algorithms Provided by Amazon SageMaker: Common Parameters' in the SageMaker documentation for an explanation of these values.
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

In [65]:
time.ctime()

'Sat May  2 13:41:27 2020'

In [63]:
time.gmtime()

time.struct_time(tm_year=2020, tm_mon=5, tm_mday=2, tm_hour=13, tm_min=40, tm_sec=57, tm_wday=5, tm_yday=123, tm_isdst=0)

In [57]:
linear_job = 'DEMO-linear-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

In [59]:
linear_job

'DEMO-linear-2020-05-02-13-39-07'

In [66]:
linear_job = 'DEMO-linear-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Job name is:", linear_job)

linear_training_params = {
    "RoleArn": role,
    "TrainingJobName": linear_job,
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.c4.2xlarge",
        "VolumeSizeInGB": 10
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train/".format(bucket, prefix),
                    "S3DataDistributionType": "ShardedByS3Key"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }

    ],
    "OutputDataConfig": {
        "S3OutputPath": "s3://{}/{}/".format(bucket, prefix)
    },
    "HyperParameters": {
        "feature_dim": "30",
        "mini_batch_size": "100",
        "predictor_type": "regressor",
        "epochs": "10",
        "num_models": "32",
        "loss": "absolute_loss"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

Job name is: DEMO-linear-2020-05-02-13-53-33


Now let's kick off our training job in SageMaker's distributed, managed training, using the parameters we just created.  Because training is managed, we don't have to wait for our job to finish to continue, but for this case, let's use boto3's 'training_job_completed_or_stopped' waiter so we can ensure that the job has been started.

In [67]:
%%time

region = boto3.Session().region_name
sm = boto3.client('sagemaker')

sm.create_training_job(**linear_training_params)

status = sm.describe_training_job(TrainingJobName=linear_job)['TrainingJobStatus']
print(status)
sm.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=linear_job)
if status == 'Failed':
    message = sm.describe_training_job(TrainingJobName=linear_job)['FailureReason']
    print('Training failed with the following error: {}'.format(message))
    raise Exception('Training job failed')

InProgress
CPU times: user 80 ms, sys: 17.2 ms, total: 97.2 ms
Wall time: 4min


---
## Host

Now that we've trained the linear algorithm on our data, let's setup a model which can later be hosted.  We will:
1. Point to the scoring container
1. Point to the model.tar.gz that came from training
1. Create the hosting model

In [73]:
linear_job

'DEMO-linear-2020-05-02-13-53-33'

In [74]:
sm.describe_training_job(TrainingJobName=linear_job)

{'TrainingJobName': 'DEMO-linear-2020-05-02-13-53-33',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-2:134482205310:training-job/demo-linear-2020-05-02-13-53-33',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://breastcancer-saiprediction/sagemaker/DEMO-breast-cancer-prediction/DEMO-linear-2020-05-02-13-53-33/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'epochs': '10',
  'feature_dim': '30',
  'loss': 'absolute_loss',
  'mini_batch_size': '100',
  'num_models': '32',
  'predictor_type': 'regressor'},
 'AlgorithmSpecification': {'TrainingImage': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'test:dcg',
    'Regex': '#quality_metric: host=\\S+, test dcg <score>=(\\S+)'},
   {'Name': 'train:progress',
    'Regex': '#progress_metric: host=\\S+, completed (\\S+) %'},
   {'Name': 'test:binary_f_beta',
    'Regex': '#quality_metric: host=\\S+, test

In [75]:
linear_hosting_container = {
    'Image': container,
    'ModelDataUrl': sm.describe_training_job(TrainingJobName=linear_job)['ModelArtifacts']['S3ModelArtifacts']
}

In [76]:
linear_hosting_container

{'Image': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:1',
 'ModelDataUrl': 's3://breastcancer-saiprediction/sagemaker/DEMO-breast-cancer-prediction/DEMO-linear-2020-05-02-13-53-33/output/model.tar.gz'}

In [79]:
linear_job,role,linear_hosting_container

('DEMO-linear-2020-05-02-13-53-33',
 'arn:aws:iam::134482205310:role/service-role/AmazonSageMaker-ExecutionRole-20200501T081013',
 {'Image': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:1',
  'ModelDataUrl': 's3://breastcancer-saiprediction/sagemaker/DEMO-breast-cancer-prediction/DEMO-linear-2020-05-02-13-53-33/output/model.tar.gz'})

In [80]:
create_model_response = sm.create_model(
    ModelName=linear_job,
    ExecutionRoleArn=role,
    PrimaryContainer=linear_hosting_container)

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-east-2:134482205310:model/demo-linear-2020-05-02-13-53-33


In [81]:
create_model_response

{'ModelArn': 'arn:aws:sagemaker:us-east-2:134482205310:model/demo-linear-2020-05-02-13-53-33',
 'ResponseMetadata': {'RequestId': '9d145148-debf-4907-a116-0aee41b9945b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9d145148-debf-4907-a116-0aee41b9945b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Sat, 02 May 2020 17:05:02 GMT'},
  'RetryAttempts': 0}}

Once we've setup a model, we can configure what our hosting endpoints should be.  Here we specify:
1. EC2 instance type to use for hosting
1. Initial number of instances
1. Our hosting model name

In [82]:
linear_endpoint_config = 'DEMO-linear-endpoint-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(linear_endpoint_config)

DEMO-linear-endpoint-config-2020-05-02-17-10-49


In [85]:

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=linear_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': linear_job,
        'VariantName': 'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint Config Arn: arn:aws:sagemaker:us-east-2:134482205310:endpoint-config/demo-linear-endpoint-config-2020-05-02-17-10-49


In [86]:
create_endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:134482205310:endpoint-config/demo-linear-endpoint-config-2020-05-02-17-10-49',
 'ResponseMetadata': {'RequestId': '60eb1465-0b50-423f-8674-85d77fd23cf3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '60eb1465-0b50-423f-8674-85d77fd23cf3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '128',
   'date': 'Sat, 02 May 2020 17:18:01 GMT'},
  'RetryAttempts': 0}}

Now that we've specified how our endpoint should be configured, we can create them.  This can be done in the background, but for now let's run a loop that updates us on the status of the endpoints so that we know when they are ready for use.

In [87]:
%%time

linear_endpoint = 'DEMO-linear-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(linear_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=linear_endpoint,
    EndpointConfigName=linear_endpoint_config)
print(create_endpoint_response['EndpointArn'])


DEMO-linear-endpoint-202005021721
arn:aws:sagemaker:us-east-2:134482205310:endpoint/demo-linear-endpoint-202005021721
CPU times: user 13.5 ms, sys: 0 ns, total: 13.5 ms
Wall time: 173 ms


In [92]:
create_endpoint_response

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:134482205310:endpoint/demo-linear-endpoint-202005021721',
 'ResponseMetadata': {'RequestId': 'fde46795-6882-44f8-87c0-ca28e5a590e1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fde46795-6882-44f8-87c0-ca28e5a590e1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Sat, 02 May 2020 17:21:18 GMT'},
  'RetryAttempts': 0}}

In [94]:
sm.describe_endpoint(EndpointName=linear_endpoint)

{'EndpointName': 'DEMO-linear-endpoint-202005021721',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:134482205310:endpoint/demo-linear-endpoint-202005021721',
 'EndpointConfigName': 'DEMO-linear-endpoint-config-2020-05-02-17-10-49',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:1',
     'ResolvedImage': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner@sha256:d9b089005a563bf0023a95c3fc7db6c46ad262f949d79d8f1bba6d97ffd21b70',
     'ResolutionTime': datetime.datetime(2020, 5, 2, 17, 21, 22, 56000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2020, 5, 2, 17, 21, 18, 967000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 5, 2, 17, 26, 27, 278000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '475

In [95]:
%%time

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

sm.get_waiter('endpoint_in_service').wait(EndpointName=linear_endpoint)

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp['EndpointStatus']
print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

if status != 'InService':
    raise Exception('Endpoint creation did not succeed')

Status: InService
Arn: arn:aws:sagemaker:us-east-2:134482205310:endpoint/demo-linear-endpoint-202005021721
Status: InService
CPU times: user 110 ms, sys: 0 ns, total: 110 ms
Wall time: 299 ms


## Predict
### Predict on Test Data

Now that we have our hosted endpoint, we can generate statistical predictions from it.  Let's predict on our test dataset to understand how accurate our model is.

There are many metrics to measure classification accuracy.  Common examples include include:
- Precision
- Recall
- F1 measure
- Area under the ROC curve - AUC
- Total Classification Accuracy 
- Mean Absolute Error

For our example, we'll keep things simple and use total classification accuracy as our metric of choice. We will also evaluate  Mean Absolute  Error (MAE) as the linear-learner has been optimized using this metric, not necessarily because it is a relevant metric from an application point of view. We'll compare the performance of the linear-learner against a naive benchmark prediction which uses majority class observed in the training data set for prediction on the test data.




In [96]:
testsai=np.random.rand(10)

In [98]:
testsai,type(testsai)

(array([0.87000325, 0.66947857, 0.47669649, 0.32862037, 0.01304296,
        0.24353486, 0.52359231, 0.6677753 , 0.97711858, 0.30685741]),
 numpy.ndarray)

### Function to convert an array to a csv

In [99]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

In [102]:
csv2=np2csv(testsai)#,

In [103]:
csv2

'0.870003\n0.669479\n0.476696\n0.32862\n0.013043\n0.243535\n0.523592\n0.667775\n0.977119\n0.306857'

Next, we'll invoke the endpoint to get predictions.

In [106]:
test_X.shape

(56, 30)

In [107]:
runtime= boto3.client('runtime.sagemaker')

payload = np2csv(test_X)
response = runtime.invoke_endpoint(EndpointName=linear_endpoint,
                                   ContentType='text/csv',
                                   Body=payload)


In [108]:
response

{'ResponseMetadata': {'RequestId': '0241224b-92ea-4b8b-8ea7-31f92dfff166',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0241224b-92ea-4b8b-8ea7-31f92dfff166',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Sat, 2 May 2020 17:36:17 GMT',
   'content-type': 'application/json',
   'content-length': '1798'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f1cc794cba8>}

In [109]:
result = json.loads(response['Body'].read().decode())

In [117]:
result

{'predictions': [{'score': 0.45492446422576904},
  {'score': 0.045729637145996094},
  {'score': -0.13833922147750854},
  {'score': 0.1424194574356079},
  {'score': 0.6725425720214844},
  {'score': 0.05433237552642822},
  {'score': 0.16156554222106934},
  {'score': 0.5196934938430786},
  {'score': 0.3986762762069702},
  {'score': 0.16748583316802979},
  {'score': 0.11325860023498535},
  {'score': 0.1983548402786255},
  {'score': 0.23363447189331055},
  {'score': 0.21502137184143066},
  {'score': 0.5871627330780029},
  {'score': 0.06336426734924316},
  {'score': 0.11362838745117188},
  {'score': 0.08752405643463135},
  {'score': 0.3580772876739502},
  {'score': 0.1907510757446289},
  {'score': 0.5700160264968872},
  {'score': 0.6812875270843506},
  {'score': 0.5837067365646362},
  {'score': -0.01910334825515747},
  {'score': 0.21486306190490723},
  {'score': 0.010578751564025879},
  {'score': 0.17079663276672363},
  {'score': 0.19078028202056885},
  {'score': 0.07973039150238037},
  {'sc

In [123]:
emp_list=[]
for i in result['predictions']:
       
    emp_list.append(i['score'])

In [126]:
saitestpredictions=np.array(emp_list)

In [127]:
saitestpredictions

array([ 0.45492446,  0.04572964, -0.13833922,  0.14241946,  0.67254257,
        0.05433238,  0.16156554,  0.51969349,  0.39867628,  0.16748583,
        0.1132586 ,  0.19835484,  0.23363447,  0.21502137,  0.58716273,
        0.06336427,  0.11362839,  0.08752406,  0.35807729,  0.19075108,
        0.57001603,  0.68128753,  0.58370674, -0.01910335,  0.21486306,
        0.01057875,  0.17079663,  0.19078028,  0.07973039,  0.06081653,
        1.15862   ,  0.07297528, -0.06769288,  0.7806462 ,  0.48911083,
       -0.14099413,  0.14560699,  0.79602146,  0.05141705,  0.07148468,
        0.61654556,  0.20432067,  0.02632582,  0.34614849,  0.145904  ,
        0.09219193,  0.22086728,  0.15553606,  0.24678338,  0.04980457,
       -0.00197208,  0.20468247,  0.04024237, -0.03060806,  0.99835038,
       -0.11420298])

In [114]:
test_pred = np.array([r['score'] for r in result['predictions']])

In [115]:
test_pred

array([ 0.45492446,  0.04572964, -0.13833922,  0.14241946,  0.67254257,
        0.05433238,  0.16156554,  0.51969349,  0.39867628,  0.16748583,
        0.1132586 ,  0.19835484,  0.23363447,  0.21502137,  0.58716273,
        0.06336427,  0.11362839,  0.08752406,  0.35807729,  0.19075108,
        0.57001603,  0.68128753,  0.58370674, -0.01910335,  0.21486306,
        0.01057875,  0.17079663,  0.19078028,  0.07973039,  0.06081653,
        1.15862   ,  0.07297528, -0.06769288,  0.7806462 ,  0.48911083,
       -0.14099413,  0.14560699,  0.79602146,  0.05141705,  0.07148468,
        0.61654556,  0.20432067,  0.02632582,  0.34614849,  0.145904  ,
        0.09219193,  0.22086728,  0.15553606,  0.24678338,  0.04980457,
       -0.00197208,  0.20468247,  0.04024237, -0.03060806,  0.99835038,
       -0.11420298])

Let's compare linear learner based mean absolute prediction errors from a baseline prediction which uses majority class to predict every instance.

In [128]:
test_mae_linear = np.mean(np.abs(test_y - test_pred))
test_mae_baseline = np.mean(np.abs(test_y - np.median(train_y))) ## training median as baseline predictor

print("Test MAE Baseline :", round(test_mae_baseline, 3))
print("Test MAE Linear:", round(test_mae_linear,3))


Test MAE Baseline : 0.268
Test MAE Linear: 0.192


Let's compare predictive accuracy using a classification threshold of 0.5 for the predicted and compare against the majority class prediction from training data set

In [129]:
test_pred_class = (test_pred > 0.5)+0;
test_pred_baseline = np.repeat(np.median(train_y), len(test_y))

prediction_accuracy = np.mean((test_y == test_pred_class))*100
baseline_accuracy = np.mean((test_y == test_pred_baseline))*100

print("Prediction Accuracy:", round(prediction_accuracy,1), "%")
print("Baseline Accuracy:", round(baseline_accuracy,1), "%")

Prediction Accuracy: 92.9 %
Baseline Accuracy: 73.2 %


###### Run the cell below to delete endpoint once you are done.

In [ ]:
sm.delete_endpoint(EndpointName=linear_endpoint)

---
## Extensions

- Our linear model does a good job of predicting breast cancer and has an overall accuracy of close to 92%. We can re-run the model with different values of the hyper-parameters, loss functions etc and see if we get improved prediction. Re-running the model with further tweaks to these hyperparameters may provide more accurate out-of-sample predictions.
- We also did not do much feature engineering. We can create additional features by considering cross-product/intreaction of multiple features, squaring or raising higher powers of the features to induce non-linear effects, etc. If we expand the features using non-linear terms and interactions, we can then tweak the regulaization parameter to optimize the expanded model and hence generate improved forecasts.
- As a further extension, we can use many of non-linear models available through SageMaker such as XGBoost, MXNet etc.
